# Skeleton of Assignment 4:

test if the distributions of 
    
### 1) trip duration of bikers that ride during the day vs night
### 2) age of bikers for trips originating in Manhattan and in Brooklyn
    
are different. Use 3 tests: KS, Pearson's, Spearman's. 
    
Use the scipy.stats functions scipy.stats.ks_2samp, scipy.stats.pearsonr, scipy.stats.spearmanr. 
    
For the KS do the test with the entire dataset and with a subset 200 times smaller
    
Choose a single significant threshold for the whole exercise. 
    
For each test phrase the Null Hypothesis in words.
    
Describe the return of the scipy function you use in each case.
    
State the result in terms of rejection of the Null.

In [53]:
# my usual imports and setups
import random 
import pylab as pl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
#imports downloader
from getCitiBikeCSV import getCitiBikeCSV
%pylab inline

import os

os.getenv("PUIDATA")
#this makes my plots pretty! but it is totally not mandatory to do it
#import json
#s = json.load( open(os.getenv ('PUI2016')+"/fbb_matplotlibrc.json") )
#pl.rcParams.update(s)



Populating the interactive namespace from numpy and matplotlib


/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['random']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


'/home/cusp/ekh331/PUIdata'

# Read in data
I am reading in data from January 2015 with the function that I created getCitiBikeCSV. You are requested to use 2 months at least. It would be a good idea to use data from a colder and a warmer months, since there are more riders in the warm weather and ridership patterns may change with weather, temperature, etc. You should use data from multiple months, joining multiple datasets (thus addressing some systematic errors as well)

In [9]:
#datestring = '201511'
#getCitiBikeCSV(datestring)

In [3]:
!wget -O - 'https://s3.amazonaws.com/tripdata/201512-citibike-tripdata.zip' > 'dec_data.zip'
!wget -O - 'https://s3.amazonaws.com/tripdata/201511-citibike-tripdata.zip' > 'nov_data.zip'

--2017-10-14 20:41:39--  https://s3.amazonaws.com/tripdata/201512-citibike-tripdata.zip
Resolving s3.amazonaws.com... 52.216.224.235
Connecting to s3.amazonaws.com|52.216.224.235|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23145783 (22M) [application/zip]
Saving to: “STDOUT”

100%[======================================>] 23,145,783  45.8M/s   in 0.5s    

2017-10-14 20:41:40 (45.8 MB/s) - written to stdout [23145783/23145783]

--2017-10-14 20:41:40--  https://s3.amazonaws.com/tripdata/201511-citibike-tripdata.zip
Resolving s3.amazonaws.com... 52.216.224.235
Connecting to s3.amazonaws.com|52.216.224.235|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34354395 (33M) [application/zip]
Saving to: “STDOUT”

100%[======================================>] 34,354,395  41.7M/s   in 0.8s    

2017-10-14 20:41:41 (41.7 MB/s) - written to stdout [34354395/34354395]



In [4]:
import zipfile

zip_dec = zipfile.ZipFile('dec_data.zip', 'r')
zip_dec.extractall()

zip_nov = zipfile.ZipFile('nov_data.zip', 'r')
zip_nov.extractall()

dec_df = pd.read_csv('201512-citibike-tripdata.csv')

dec = dec_df

nov_df = pd.read_csv('201511-citibike-tripdata.csv')

nov = nov_df

In [5]:
nov.head(1)

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,1110,11/1/2015 00:00:00,11/1/2015 00:18:31,537,Lexington Ave & E 24 St,40.740259,-73.984092,531,Forsyth St & Broome St,40.718939,-73.992663,22545,Subscriber,1981.0,2


In [6]:
#combine the two months of data

months = pd.concat([nov, dec])
print nov.shape, dec.shape, months.shape

(987245, 15) (804125, 15) (1791370, 15)


In [7]:
# df is the dataframe where the content of the csv file is stored
months['date'] = pd.to_datetime(months['starttime'])
# note that with dataframes I can refer to variables as dictionary keys, 
# i.e. df['starttime'] or as attributes: df.starttime. 
months.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,date
0,1110,11/1/2015 00:00:00,11/1/2015 00:18:31,537,Lexington Ave & E 24 St,40.740259,-73.984092,531,Forsyth St & Broome St,40.718939,-73.992663,22545,Subscriber,1981.0,2,2015-11-01 00:00:00
1,1094,11/1/2015 00:00:01,11/1/2015 00:18:15,537,Lexington Ave & E 24 St,40.740259,-73.984092,531,Forsyth St & Broome St,40.718939,-73.992663,23959,Subscriber,1980.0,1,2015-11-01 00:00:01
2,520,11/1/2015 00:00:05,11/1/2015 00:08:45,536,1 Ave & E 30 St,40.741444,-73.975361,498,Broadway & W 32 St,40.748549,-73.988084,22251,Subscriber,1988.0,1,2015-11-01 00:00:05
3,753,11/1/2015 00:00:15,11/1/2015 00:12:48,229,Great Jones St,40.727434,-73.993790,328,Watts St & Greenwich St,40.724055,-74.009660,15869,Subscriber,1981.0,1,2015-11-01 00:00:15
4,353,11/1/2015 00:00:22,11/1/2015 00:06:15,285,Broadway & E 14 St,40.734546,-73.990741,151,Cleveland Pl & Spring St,40.722104,-73.997249,21645,Subscriber,1987.0,1,2015-11-01 00:00:22


In [13]:
original_months = months

In [109]:
months = original_months
months.head(5)
print(months.shape)

(1791370, 16)


In [110]:
# Remove all rows with NaN birthyear
months = months.dropna(subset = ['birth year'])
print(months.shape)
print(months.head(5))

(1622892, 16)
   tripduration           starttime            stoptime  start station id  \
0          1110  11/1/2015 00:00:00  11/1/2015 00:18:31               537   
1          1094  11/1/2015 00:00:01  11/1/2015 00:18:15               537   
2           520  11/1/2015 00:00:05  11/1/2015 00:08:45               536   
3           753  11/1/2015 00:00:15  11/1/2015 00:12:48               229   
4           353  11/1/2015 00:00:22  11/1/2015 00:06:15               285   

        start station name  start station latitude  start station longitude  \
0  Lexington Ave & E 24 St               40.740259               -73.984092   
1  Lexington Ave & E 24 St               40.740259               -73.984092   
2          1 Ave & E 30 St               40.741444               -73.975361   
3           Great Jones St               40.727434               -73.993790   
4       Broadway & E 14 St               40.734546               -73.990741   

   end station id          end station name  end

In [111]:
def do(problem_number):
    """ 
    problem_number is either 1 or 2, corresponding to:
        1: Test whether trip durations of bikers is different during day vs night
        2: Test whether age of bikers is different for Manhattan vs Brooklyn
        
    Each problem tests whether the distribution of feature_of_interest (duration or age) is 
    different for type_1 vs type_2 (day vs night, or Manhattan vs Brooklyn)
    """
    
    print("Doing problem %s..." % problem_number)
    
    type_1_rows, type_2_rows = identify_types(problem_number) 
    feature_column = feature_of_interest(problem_number)
    
    sample_1 = feature_column[type_1_rows]
    sample_2 = feature_column[type_2_rows]
    
    print("First few values of type 1")
    print(sample_1.head(5))
    print("First few values of type 2")
    print(sample_2.head(5))
    
    alpha = 0.05
    
    # KS
    ks_stat, p = scipy.stats.ks_2samp(sample_1, sample_2)
    print(ks_null_hypothesis(problem_number))
    print("p_value for KS: %s" % p)
    print(compare_p_to_alpha(p, alpha))
    
    # Let's make smaller samples for a reduced KS. Probably we will see a higher p-value.
    random.seed(1)
    sample_1_copy, sample_2_copy = [d for d in sample_1], [d for d in sample_2]
    random.shuffle(sample_1_copy)
    random.shuffle(sample_2_copy)
    smaller_sample_1 = sample_1_copy[::20]
    smaller_sample_2 = sample_2_copy[::20]
    print("KS on smaller samples (1/20th the size):")
    ks_stat, p = scipy.stats.ks_2samp(smaller_sample_1, smaller_sample_2)
    print(ks_null_hypothesis(problem_number))
    print("p_value for KS: %s" % p)
    print(compare_p_to_alpha(p, alpha))
    
    
    # We will need equal-length samples, so let's resample...
    new_size = min(len(sample_1), len(sample_2))
    print("For Pearson, reducing both samples to new_size: %s" % new_size)
    new_sample_1 = sample_1_copy[:new_size]
    new_sample_2 = sample_2_copy[:new_size]
    
    # Pearson
    rho, p = scipy.stats.pearsonr(new_sample_1, new_sample_2)
    print(pearsonnr_null_hypothesis(problem_number))
    print("p_value for Pearson: %s" % p)
    print(compare_p_to_alpha(p, alpha))
    
    # Spearmanr
    rho, p = scipy.stats.spearmanr(new_sample_1, new_sample_2)
    print(spearmannr_null_hypothesis(problem_number))
    print("p_value for Spearman: %s" % p)
    print(compare_p_to_alpha(p, alpha))

    
def compare_p_to_alpha(p, alpha):
    if p < alpha:
        return "p < alpha, so we reject the null hypothesis."
    else:
        return "p >= alpha, so we do not reject the null hypothesis."
    
def feature_of_interest(problem_number):
    """ Returns a column of values, with the same number of rows as the months dataframe. """
    if problem_number == 1:
        return months['tripduration']
    if problem_number == 2:
        return 2015 - months['birth year']
    
def identify_types(problem_number):
    """
    Returns (type_1_rows, type_2_rows), each of which is a column of True/False where Trues 
    correspond to rows in months of that type.
    """
    
    if problem_number == 1:
        # Types are day and night
        is_day = lambda d: 5 <= d.hour <= 19
        is_night = lambda d: not is_day(d)
        
        type_1_rows = months['date'].apply(is_day)
        type_2_rows = months['date'].apply(is_night)
    elif problem_number == 2:
        # Types are Manhattan and Brooklyn
        # We'll choose coordinates of two points, and declare everything above the corresponding line 
        # to be Manhattan, and everything below is Brooklyn.
        
        bottom_left = (40.697409, -74.011974)
        top_right = (40.721091, -73.969574)
        
        # Imagine vectors a, (bottom_left -> top_right), and b, (top_right -> new_point)
        # If the cross product a x b is positive, the new point is in Manhattan
        a = (top_right[0] - bottom_left[0], 
             top_right[1] - bottom_left[1])
        b = (months['start station latitude'] - top_right[0],
             months['start station longitude'] - top_right[1])
        
        cross_product = a[0] * b[1] - a[1] * b[0]
        type_1_rows = cross_product > 0
        type_2_rows = cross_product < 0
    
    return (type_1_rows, type_2_rows)


##### Null hypotheses ##########################################################################
def ks_null_hypothesis(problem_number):
    prefix = "The null hypothesis for KS is that the distribution of "
    if problem_number == 1:
        suffix = "trip duration is the same during the day vs during the night."
    if problem_number == 2:
        suffix = "age is the same for trips originating in Manhattan vs Brooklyn."
        
    return prefix + suffix

def pearsonnr_null_hypothesis(problem_number):
    prefix = "The null hypothesis for Pearson is that there is 0 correlation between"
    if problem_number == 1:
        suffix = "trip duration and an indicator variable for day vs night." 
    if problem_number == 2:
        suffix = "age and an indicator variable for originating in Manhattan vs Brooklyn."
        
    return prefix + suffix

def spearmannr_null_hypothesis(problem_number):
    prefix = "The null hypothesis for Spearman is that there is no association between "
    if problem_number == 1:
        suffix = "trip duration and an indicator variable for day vs night."
    if problem_number == 2:
        suffix = "age and an indicator variable for originating in Manhattan vs Brooklyn."
    
    return prefix + suffix

In [112]:
do(1)

Doing problem 1...
First few values of type 1
1911    9668
1912     790
1913     555
1914    1805
1915     466
Name: tripduration, dtype: int64
First few values of type 2
0    1110
1    1094
2     520
3     753
4     353
Name: tripduration, dtype: int64
The null hypothesis for KS is that the distribution of trip duration is the same during the day vs during the night.
p_value for KS: 2.36667977115e-282
p < alpha, so we reject the null hypothesis.
KS on smaller samples (1/20th the size):
The null hypothesis for KS is that the distribution of trip duration is the same during the day vs during the night.
p_value for KS: 6.52071464349e-24
p < alpha, so we reject the null hypothesis.
For Pearson, reducing both samples to new_size: 207412
The null hypothesis for Pearson is that there is 0 correlation betweentrip duration and an indicator variable for day vs night.
p_value for Pearson: 0.818000030283
p >= alpha, so we do not reject the null hypothesis.
The null hypothesis for Spearman is that

In [113]:
do(2)

Doing problem 2...
First few values of type 1
7     37.0
21    23.0
23    24.0
38    31.0
49    23.0
Name: birth year, dtype: float64
First few values of type 2
0    34.0
1    35.0
2    27.0
3    34.0
4    28.0
Name: birth year, dtype: float64
The null hypothesis for KS is that the distribution of age is the same for trips originating in Manhattan vs Brooklyn.
p_value for KS: 0.0
p < alpha, so we reject the null hypothesis.
KS on smaller samples (1/20th the size):
The null hypothesis for KS is that the distribution of age is the same for trips originating in Manhattan vs Brooklyn.
p_value for KS: 1.04440039013e-110
p < alpha, so we reject the null hypothesis.
For Pearson, reducing both samples to new_size: 150348
The null hypothesis for Pearson is that there is 0 correlation betweenage and an indicator variable for originating in Manhattan vs Brooklyn.
p_value for Pearson: 0.301663339752
p >= alpha, so we do not reject the null hypothesis.
The null hypothesis for Spearman is that there

Most of the output required for this problem, for each question 1 and 2, including null hypothesis, p-value, and interpretation, is above.

However, I want to include a description of the output of the scipy functions.

stats.pearsonr takes two arrays and returns two values: pearson's correlation coefficient (in the range of -1 to +1, where 1 = perfectly positively correlated, 0 = not correlated, and -1 = perfectly negatively correlated), and also a two-tailed p-value.

stats.spearmanr returns a rho, also known as the spearman correlation rank order coefficient (in the range of -1 to +1, where 1 = perfectly positively correlated, 0 = not correlated, and -1 = perfectly negatively correlated), and also a two-sided p-value. It does not assume that the data are normally distributed. 

stats.ks_2samp returns the KS statistic and also a p-value as floats. A small KS statistic or a large p-value prevents us from rejecting the null hypothesis.